In [1]:
from Dataset_Loader import load_leafs_dataset
from Training import train_cnn
from torch import nn, optim, manual_seed

d:\Downloads\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 20
test_split_size = 0.2

images_train, images_test, labels_train, labels_test = load_leafs_dataset(
    test_split_size,
    batch_size 
)

Reading CSV: 0.254 seconds
Encoding Labels: 0.034 seconds
Resizing Images: 11.974 seconds
Splitting Data: 0.851 seconds
Converting To Tensor: 12.139 seconds
Dividing To Batches: 0.275 seconds


In [3]:
input_images = images_train[:]
output_labels = labels_train[:]

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)

        self.fc1 = nn.Linear(16*49*32, 1000)
        self.fc2 = nn.Linear(1000, 99)

    def forward(self, x):
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.max_pool2d(x, 2, 2)

        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)

        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)

        x = x.view(-1, 16*49*32)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.log_softmax(self.fc2(x), dim=1)

        return x

In [5]:
manual_seed(1)
model = CNN()
trained_model, train_losses = train_cnn(
    model=model,
    input_tensor=input_images,
    output_tensor=output_labels,
    epochs=20,
    batch_size=batch_size,
    lossFunction=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(model.parameters(), lr=0.01),
    print_loss=True    
)

Epoch: 0 / loss: 4.4444355964660645
Epoch: 1 / loss: 4.368679046630859
Epoch: 2 / loss: 3.790904998779297
Epoch: 3 / loss: 2.2527310848236084
Epoch: 4 / loss: 1.1746814250946045
Epoch: 5 / loss: 1.0820286273956299
Epoch: 6 / loss: 0.9706239700317383
Epoch: 7 / loss: 0.9274231791496277
Epoch: 8 / loss: 4.791457653045654
Epoch: 9 / loss: 2.871070146560669
Epoch: 10 / loss: 1.8090314865112305
Epoch: 11 / loss: 1.0705147981643677
Epoch: 12 / loss: 1.1389200687408447
Epoch: 13 / loss: 1.0580593347549438
Epoch: 14 / loss: 0.8581987619400024
Epoch: 15 / loss: 1.0203227996826172
Epoch: 16 / loss: 1.2720297574996948
Epoch: 17 / loss: 0.5969955325126648
Epoch: 18 / loss: 0.7676581144332886
Epoch: 19 / loss: 0.20788168907165527
